In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision
import torchvision.transforms as transforms
from torchsummary import summary
import seaborn as sns

from PIL import Image
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import sys

test_dir = "/home/lordgrim/Work/Courses/CS6910/A1/5/test/"
img_h,img_w = 84,84

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3)
        self.conv1a = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)

        self.pool = nn.MaxPool2d(kernel_size=3, stride=2)

        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.conv2a = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3)

        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3)
        self.conv3a = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3)


        self.fc1 = nn.Linear(in_features=256, out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=256)
        self.fc3 = nn.Linear(in_features=256, out_features=33)      # change out_features according to number of classes

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv1a(x)))

        x = F.relu(self.conv2(x))
        x = self.pool(self.conv2a(x))

        x = F.relu(self.conv3(x))
        x = self.pool(self.conv3a(x))

        x = F.avg_pool2d(x, kernel_size=x.shape[2:])

        x = x.view(x.shape[0], -1)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        
path_model = "/home/lordgrim/Work/Courses/CS6910/A1/models/62_nice.pth"
net = Net()
print(net)

# transfer the model to GPU
if torch.cuda.is_available():
    net = net.cuda()
    
net.load_state_dict(torch.load(path_model))
summary(net.cuda(),(3,img_h,img_w))

In [ ]:
loader = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5, 0.5, 0.5])])

def image_loader(image):
    """load image, returns cuda tensor"""
    image = Image.open(image)
    np_img = np.array(image)
    image = loader(image).float()
    return image.cuda(), np_img  #assumes that you're using GPU

In [ ]:
## For layer 1
max_act = {0:[],1:[],2:[]}

img_patch_all = {0:[],1:[],2:[]}

for class_label in os.listdir(test_dir):
    print(class_label)
    for img_path in os.listdir(test_dir + class_label):
        activation = {}
        def get_activation(name):
            def hook(model, input, output):
                activation[name] = output.detach()
            return hook
        net.conv1.register_forward_hook(get_activation('conv1'))
        inp, img = image_loader(test_dir+class_label+"/"+img_path)

        out = net(torch.reshape(inp, (-1, 3, img_h, img_w) ))
        act = activation['conv1'].squeeze().cpu()

        # Receptive field
        # We are using square kernel only, so x=y
        rc_field = net.conv1.kernel_size[0] 

        act_10= act[9,:,:] # 10nd filter
        act_17 = act[16,:,:] # 6th filter
        act_31 = act[30,:,:] #31st filter

        for j,act_val in enumerate([act_10,act_17,act_31]):

            ## To get the top5 max in the array, we need to flatten and then find
            top5 = torch.topk(torch.flatten(act_val),5)
            max_vals = top5[0]
            ans = top5[1]

            max_x = ans%act_val.shape[0]
            max_y = ans//act_val.shape[1]

            
            
            for i,(x,y) in enumerate(zip(max_x,max_y)):
                
                x, y = int(x.numpy()), int(y.numpy())

                x1 = x 
                x2 = x + rc_field
                y1 = y 
                y2 = y + rc_field


                img_patch_d = [test_dir+class_label+"/"+img_path,(x1,x2,y1,y2)]

                max_act[j].append(max_vals[i])
                img_patch_all[j].append(img_patch_d)

        # plt.show()



In [ ]:
for j,layer in enumerate(['layer_10','layer_17','layer_31']):
    # fig = plt.figure(figsize=(10,10))
    plt.figure(figsize=(10,10))
    activations = np.array(max_act[j])
    img_patches = img_patch_all[j]

    temp = np.argpartition(-activations, 5)
    result_args = temp[:5]
    print(result_args)

    f, axarr = plt.subplots(2,5, figsize=(15,15))
    for i,index in enumerate(result_args):
        vals = img_patches[index]
        img = np.array(Image.open(vals[0]))
        x1, x2, y1, y2 = vals[1]
        img_patch = img[y1:y2,x1:x2]
        # ax = fig.add_subplot(3, 5, 5*j + i + 1)

        axarr[0,i].axis('off',aspect="auto")
        axarr[0,i].imshow(img)
        axarr[1,i].axis('off',aspect="auto")
        axarr[1,i].imshow(img_patch)

        plt.axis('off')
    plt.show()
    f.savefig('conv1_{}.png'.format(layer))
    

In [ ]:
## For 2nd conv layer
max_act = {0:[],1:[],2:[]}

img_patch_all = {0:[],1:[],2:[]}

for class_label in os.listdir(test_dir):
    print(class_label)
    for img_path in os.listdir(test_dir + class_label):

        activation = {}
        def get_activation(name):
            def hook(model, input, output):
                activation[name] = output.detach()
            return hook
        net.conv1a.register_forward_hook(get_activation('conv1a'))
        inp, img = image_loader(test_dir+class_label+"/"+img_path)

        out = net(torch.reshape(inp, (-1, 3, img_h, img_w) ))
        act = activation['conv1a'].squeeze().cpu()

        # Receptive field
        # We are using square kernel only, so x=y

        act_1= act[18,:,:] # 10nd filter
        act_2 = act[2,:,:] # 6th filter
        act_3 = act[3,:,:] # 61th filter

        for j,act_val in enumerate([act_1,act_2,act_3]):

            ## To get the top5 max in the array, we need to flatten and then find
            top5 = torch.topk(torch.flatten(act_val),5)
            max_vals = top5[0]
            ans = top5[1]

            max_x = ans%act_val.shape[0]
            max_y = ans//act_val.shape[1]

            
            
            for i,(x,y) in enumerate(zip(max_x,max_y)):
                
                x, y = int(x.numpy()), int(y.numpy())

                x1 = x 
                x2 = x+ 5
                y1 = y 
                y2 = y+ 5


                img_patch_d = [test_dir+class_label+"/"+img_path,(x1,x2,y1,y2)]

                max_act[j].append(max_vals[i])
                img_patch_all[j].append(img_patch_d)

        # plt.show()



In [ ]:
for j,layer in enumerate(['layer_10','layer_6','layer_61']):
    # fig = plt.figure(figsize=(10,10))
    plt.figure(figsize=(10,10))
    activations = np.array(max_act[j])
    img_patches = img_patch_all[j]

    temp = np.argpartition(-activations, 5)
    result_args = temp[:5]
    print(result_args)

    f, axarr = plt.subplots(2,5, figsize=(15,15))
    for i,index in enumerate(result_args):
        vals = img_patches[index]
        img = np.array(Image.open(vals[0]))
        x1, x2, y1, y2 = vals[1]
        img_patch = img[y1:y2,x1:x2]
        # ax = fig.add_subplot(3, 5, 5*j + i + 1)

        axarr[0,i].axis('off',aspect="auto")
        axarr[0,i].imshow(img)
        axarr[1,i].axis('off',aspect="auto")
        axarr[1,i].imshow(img_patch)

        plt.axis('off')
    plt.show()
    f.savefig('conv2_{}.png'.format(layer))

    

In [ ]:
## For 3nd conv layer
max_act = {0:[],1:[],2:[]}

img_patch_all = {0:[],1:[],2:[]}

for class_label in os.listdir(test_dir):
    print(class_label)
    for img_path in os.listdir(test_dir + class_label):

        activation = {}
        def get_activation(name):
            def hook(model, input, output):
                activation[name] = output.detach()
            return hook
        net.conv2.register_forward_hook(get_activation('conv2'))
        inp, img = image_loader(test_dir+class_label+"/"+img_path)

        out = net(torch.reshape(inp, (-1, 3, img_h, img_w) ))
        act = activation['conv2'].squeeze().cpu()

        # Receptive field
        # We are using square kernel only, so x=y

        act_1= act[6,:,:] # 10nd filter
        act_2 = act[35,:,:] # 6th filter
        act_3 = act[58,:,:] # 61th filter

        for j,act_val in enumerate([act_1,act_2,act_3]):

            ## To get the top5 max in the array, we need to flatten and then find
            top5 = torch.topk(torch.flatten(act_val),5)
            max_vals = top5[0]
            ans = top5[1]

            max_x = ans%act_val.shape[0]
            max_y = ans//act_val.shape[1]

            
            
            for i,(x,y) in enumerate(zip(max_x,max_y)):
                
                x, y = int(x.numpy()), int(y.numpy())

                x1 = 2*x 
                x2 = 2*x + 11
                y1 = 2*y 
                y2 = 2*y + 11


                img_patch_d = [test_dir+class_label+"/"+img_path,(x1,x2,y1,y2)]

                max_act[j].append(max_vals[i])
                img_patch_all[j].append(img_patch_d)

        # plt.show()



In [ ]:
for j,layer in enumerate(['layer_10','layer_6','layer_61']):
    # fig = plt.figure(figsize=(10,10))
    plt.figure(figsize=(10,10))
    activations = np.array(max_act[j])
    img_patches = img_patch_all[j]

    temp = np.argpartition(-activations, 5)
    result_args = temp[:5]
    print(result_args)

    f, axarr = plt.subplots(2,5, figsize=(15,15))
    for i,index in enumerate(result_args):
        vals = img_patches[index]
        img = np.array(Image.open(vals[0]))
        x1, x2, y1, y2 = vals[1]
        img_patch = img[y1:y2,x1:x2]
        # ax = fig.add_subplot(3, 5, 5*j + i + 1)

        axarr[0,i].axis('off',aspect="auto")
        axarr[0,i].imshow(img)
        axarr[1,i].axis('off',aspect="auto")
        axarr[1,i].imshow(img_patch)

        plt.axis('off')
    plt.show()
    f.savefig('conv3_{}.png'.format(layer))

    

In [ ]:
## For 4th conv layer
max_act = {0:[],1:[],2:[]}

img_patch_all = {0:[],1:[],2:[]}

for class_label in os.listdir(test_dir):
    print(class_label)
    for img_path in os.listdir(test_dir + class_label):

        activation = {}
        def get_activation(name):
            def hook(model, input, output):
                activation[name] = output.detach()
            return hook
        net.conv2a.register_forward_hook(get_activation('conv2a'))
        inp, img = image_loader(test_dir+class_label+"/"+img_path)

        out = net(torch.reshape(inp, (-1, 3, img_h, img_w) ))
        act = activation['conv2a'].squeeze().cpu()

        # Receptive field
        # We are using square kernel only, so x=y

        act_1= act[3,:,:] # 10nd filter
        act_2 = act[1,:,:] # 6th filter
        act_3 = act[15,:,:] # 61th filter

        for j,act_val in enumerate([act_1,act_2,act_3]):

            ## To get the top5 max in the array, we need to flatten and then find
            top5 = torch.topk(torch.flatten(act_val),5)
            max_vals = top5[0]
            ans = top5[1]

            max_x = ans%act_val.shape[0]
            max_y = ans//act_val.shape[1]

            
            
            for i,(x,y) in enumerate(zip(max_x,max_y)):
                
                x, y = int(x.numpy()), int(y.numpy())

                x1 = 2*x 
                x2 = 2*x + 15
                y1 = 2*y 
                y2 = 2*y + 15


                img_patch_d = [test_dir+class_label+"/"+img_path,(x1,x2,y1,y2)]

                max_act[j].append(max_vals[i])
                img_patch_all[j].append(img_patch_d)

        # plt.show()



In [ ]:
for j,layer in enumerate(['layer_10','layer_6','layer_61']):
    # fig = plt.figure(figsize=(10,10))
    plt.figure(figsize=(10,10))
    activations = np.array(max_act[j])
    img_patches = img_patch_all[j]

    temp = np.argpartition(-activations, 5)
    result_args = temp[:5]
    print(result_args)

    f, axarr = plt.subplots(2,5, figsize=(15,15))
    for i,index in enumerate(result_args):
        vals = img_patches[index]
        img = np.array(Image.open(vals[0]))
        x1, x2, y1, y2 = vals[1]
        img_patch = img[y1:y2,x1:x2]
        # ax = fig.add_subplot(3, 5, 5*j + i + 1)

        axarr[0,i].axis('off',aspect="auto")
        axarr[0,i].imshow(img)
        axarr[1,i].axis('off',aspect="auto")
        axarr[1,i].imshow(img_patch)

        plt.axis('off')
    plt.show()
    f.savefig('conv4_{}.png'.format(layer))

    

In [ ]:
## For 5th conv layer
max_act = {0:[],1:[],2:[]}

img_patch_all = {0:[],1:[],2:[]}

for class_label in os.listdir(test_dir):
    print(class_label)
    for img_path in os.listdir(test_dir + class_label):

        activation = {}
        def get_activation(name):
            def hook(model, input, output):
                activation[name] = output.detach()
            return hook
        net.conv3.register_forward_hook(get_activation('conv3'))
        inp, img = image_loader(test_dir+class_label+"/"+img_path)

        out = net(torch.reshape(inp, (-1, 3, img_h, img_w) ))
        act = activation['conv3'].squeeze().cpu()
        # print(act.shape)

        # Receptive field
        # We are using square kernel only, so x=y

        act_1= act[200,:,:] # 10nd filter
        act_2 = act[35,:,:] # 6th filter
        act_3 = act[95,:,:] # 61th filter

        for j,act_val in enumerate([act_1,act_2,act_3]):

            ## To get the top5 max in the array, we need to flatten and then find
            top5 = torch.topk(torch.flatten(act_val),5)
            max_vals = top5[0]
            ans = top5[1]

            max_x = ans%act_val.shape[0]
            max_y = ans//act_val.shape[1]

            
            
            for i,(x,y) in enumerate(zip(max_x,max_y)):
                
                x, y = int(x.numpy()), int(y.numpy())

                x1 = 4*x 
                x2 = 4*x + 25
                y1 = 4*y 
                y2 = 4*y + 25


                img_patch_d = [test_dir+class_label+"/"+img_path,(x1,x2,y1,y2)]

                max_act[j].append(max_vals[i])
                img_patch_all[j].append(img_patch_d)

        # plt.show()



In [ ]:
for j,layer in enumerate(['layer_10','layer_6','layer_61']):
    # fig = plt.figure(figsize=(10,10))
    plt.figure(figsize=(10,10))
    activations = np.array(max_act[j])
    img_patches = img_patch_all[j]

    temp = np.argpartition(-activations, 5)
    result_args = temp[:5]
    print(result_args)

    f, axarr = plt.subplots(2,5, figsize=(15,15))
    for i,index in enumerate(result_args):
        vals = img_patches[index]
        img = np.array(Image.open(vals[0]))
        x1, x2, y1, y2 = vals[1]
        img_patch = img[y1:y2,x1:x2]
        print(x1, x2, y1, y2)
        # ax = fig.add_subplot(3, 5, 5*j + i + 1)

        axarr[0,i].axis('off',aspect="auto")
        axarr[0,i].imshow(img)
        axarr[1,i].axis('off',aspect="auto")
        axarr[1,i].imshow(img_patch)

        plt.axis('off')
    plt.show()
    f.savefig('conv5_{}.png'.format(layer))

    